In [1]:
!git clone -b pose-extraction https://github.com/YUyou0/ict3104-team14-2023.git

Cloning into 'ict3104-team14-2023'...
remote: Enumerating objects: 85, done.
remote: Counting objects: 100% (29/29), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 85 (delta 10), reused 21 (delta 4), pack-reused 56
Receiving objects: 100% (85/85), 57.13 MiB | 27.66 MiB/s, done.
Resolving deltas: 100% (20/20), done.


In [2]:
%cd /content/ict3104-team14-2023/

import cv2 as cv
import numpy as np
from google.colab.patches import cv2_imshow

BODY_PARTS = { "Nose": 0, "Neck": 1, "RShoulder": 2, "RElbow": 3, "RWrist": 4,
               "LShoulder": 5, "LElbow": 6, "LWrist": 7, "RHip": 8, "RKnee": 9,
               "RAnkle": 10, "LHip": 11, "LKnee": 12, "LAnkle": 13, "REye": 14,
               "LEye": 15, "REar": 16, "LEar": 17, "Background": 18 }

POSE_PAIRS = [ ["Neck", "RShoulder"], ["Neck", "LShoulder"], ["RShoulder", "RElbow"],
               ["RElbow", "RWrist"], ["LShoulder", "LElbow"], ["LElbow", "LWrist"],
               ["Neck", "RHip"], ["RHip", "RKnee"], ["RKnee", "RAnkle"], ["Neck", "LHip"],
               ["LHip", "LKnee"], ["LKnee", "LAnkle"], ["Neck", "Nose"], ["Nose", "REye"],
               ["REye", "REar"], ["Nose", "LEye"], ["LEye", "LEar"] ]

width = 368
height = 368
inWidth = width
inHeight = height

net = cv.dnn.readNetFromTensorflow("graph_opt.pb")
thr = 0.2

def poseDetector(frame):
    frameWidth = frame.shape[1]
    frameHeight = frame.shape[0]

    net.setInput(cv.dnn.blobFromImage(frame, 1.0, (inWidth, inHeight), (127.5, 127.5, 127.5), swapRB=True, crop=False))
    out = net.forward()
    out = out[:, :19, :, :]  # MobileNet output [1, 57, -1, -1], we only need the first 19 elements

    assert(len(BODY_PARTS) == out.shape[1])

    points = []
    for i in range(len(BODY_PARTS)):
        # Slice heatmap of corresponging body's part.
        heatMap = out[0, i, :, :]

        _, conf, _, point = cv.minMaxLoc(heatMap)
        x = (frameWidth * point[0]) / out.shape[3]
        y = (frameHeight * point[1]) / out.shape[2]
        points.append((int(x), int(y)) if conf > thr else None)

    for pair in POSE_PAIRS:
        partFrom = pair[0]
        partTo = pair[1]
        assert(partFrom in BODY_PARTS)
        assert(partTo in BODY_PARTS)

        idFrom = BODY_PARTS[partFrom]
        idTo = BODY_PARTS[partTo]

        if points[idFrom] and points[idTo]:
            cv.line(frame, points[idFrom], points[idTo], (0, 255, 0), 3)
            cv.ellipse(frame, points[idFrom], (3, 3), 0, 0, 360, (0, 0, 255), cv.FILLED)
            cv.ellipse(frame, points[idTo], (3, 3), 0, 0, 360, (0, 0, 255), cv.FILLED)

    t, _ = net.getPerfProfile()

    return frame

/content/ict3104-team14-2023


# Select a video for pose extraction

In [4]:
import os
from IPython.display import display, HTML
import ipywidgets as widgets

# Specify the folder path where you want to list files
folder_path = '/content/ict3104-team14-2023/input_videos/charades'  # Change this to your desired folder path

# Get a list of files in the folder
file_list = os.listdir(folder_path)
file_list.insert(0,"")  # Insert "" at the start of the list

# Create a dropdown widget with file options
file_dropdown = widgets.Dropdown(
    options=file_list,
    description='Select a file:',
)

# Display the dropdown widget
display(file_dropdown)

# Function to handle file selection
def select_file(change):
    global selected_file
    selected_file = change.new
    print(f"Selected file: {selected_file}")
    # Store the selected file name in a variable or use it as needed
    # For example, you can assign it to a variable like this:
    # selected_file_name = selected_file

# Attach the select_file function to the dropdown's observe method
file_dropdown.observe(select_file, names='value')

Dropdown(description='Select a file:', options=('', '5D3X6.mp4', '5O0YS.mp4', '5EFQ7.mp4', '2E8GU.mp4', 'rahee…

Selected file: raheem_dance.mp4


In [7]:
print(selected_file)
selected_file_trimmed = selected_file[:-4]
print(selected_file_trimmed)

raheem_dance.mp4
raheem_dance


# Run the pose extraction process

In [6]:
%cd /content/ict3104-team14-2023/input_videos/charades/
!pwd

import cv2
cap = cv2.VideoCapture(selected_file)
ret, frame = cap.read()
frame_height, frame_width, _ = frame.shape
out = cv2.VideoWriter(f'{selected_file_trimmed}_pose.avi',cv2.VideoWriter_fourcc('M','J','P','G'), 10, (frame_width,frame_height))
print("Processing Video...")
while cap.isOpened():
  ret, frame = cap.read()
  if not ret:
    out.release()
    break
  output = poseDetector(frame)
  out.write(output)
out.release()
print("Done processing video")

/content/ict3104-team14-2023/input_videos/charades
/content/ict3104-team14-2023/input_videos/charades
Processing Video...
Done processing video


# Remove the video background leaving only the pose extraction

In [10]:
%cd /content/ict3104-team14-2023/input_videos/charades/
!pwd

import cv2
import numpy as np

# Load the video
cap = cv2.VideoCapture('raheem_dance_pose.avi')  # Replace 'input_video.mp4' with your video file
output_video_path = 'raheem_dance_pose_extracted.mov'
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
out = cv2.VideoWriter(output_video_path, cv2.VideoWriter_fourcc(*'mp4v'), 30, (frame_width, frame_height), isColor=True)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Convert the frame to HSV color space
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # Define the lower and upper bounds for bright green and red in HSV color space
    lower_green = np.array([35, 100, 250])
    upper_green = np.array([85, 255, 255])

    lower_red1 = np.array([0, 100, 250])
    upper_red1 = np.array([80, 255, 255])

    # Create masks to isolate bright green and red regions
    mask_green = cv2.inRange(hsv, lower_green, upper_green)
    mask_red1 = cv2.inRange(hsv, lower_red1, upper_red1)

    # Combine the green and red masks
    mask_combined = cv2.bitwise_or(mask_green, mask_red1)

    # Create an all-black frame
    black_background = np.zeros_like(frame)

    # Set the pixels in the black background frame to the original frame where the mask is white
    black_background[mask_combined > 0] = frame[mask_combined > 0]

    # Write the frame to the output video
    out.write(black_background)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
out.release()
cv2.destroyAllWindows()
print("Pose extraction processing done")

/content/ict3104-team14-2023/input_videos/charades
/content/ict3104-team14-2023/input_videos/charades
Pose extraction processing done
